In [32]:
from transformers import AutoModel, AutoTokenizer
import torch
import os
import numpy as np
import torch.nn.functional as F
from einops import rearrange
import glob
from pdf2image import convert_from_path
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.utils import ImageReader
import math
import shutil
import re
import markdown2
import imgkit
from fpdf import FPDF
from PIL import Image
import subprocess
from pathlib import Path
import csv

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
model_name = 'deepseek-ai/DeepSeek-OCR'

# 加载模型和分词器
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, _attn_implementation='flash_attention_2',
                                  trust_remote_code=True, use_safetensors=True)
model = model.eval().cuda().to(torch.bfloat16)

You are using a model of type deepseek_vl_v2 to instantiate a model of type DeepseekOCR. This is not supported for all configurations of models and can yield errors.
Some weights of DeepseekOCRForCausalLM were not initialized from the model checkpoint at deepseek-ai/DeepSeek-OCR and are newly initialized: ['model.vision_model.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [56]:
DATASET_PDF_PATH = "data/MMLongBench-Doc/data/documents/"  # 数据集中pdf所在路径
RESIZED_IMGAE_PATH = "resized_image/"                      # 把数据集pdf逐页拆分+resize/padding后的img路径
MARKDOWN_OUTPUT = "markdown_output/"                       # OCR生成mmd文件结果保存路径
OUTPUT_PDF = "pdf_output/"                                 # mmd生成的pdf存放路径

WKHTMLTOIMAGE_PATH = "wkhtmltoimage"
TARGET_RESIZE_IMAGE_RESOLUTION = 1024    # 目标页面分辨率
TARGET_TOKENS_LENGTH = 256        # 目标Vision Tokens长度（H_patch × W_patch）
PATCH_SIZE = 16                   # DeepSeek-OCR默认Patch Size（固定为16）
BASE_SIZE = 1024                  # 模型基础输入尺寸
IMAGE_SIZE = 640                  # 模型实际输入尺寸（控制resize上限）Gundam
CROP_MODE = True                  # 模型裁剪模式（与原推理逻辑一致）
SAVE_OCR_RESULTS = True           # 是否保存OCR文本结果
TEST_COMPRESS = True              # 是否开启压缩模式
IMAGE_WIDTH = 600  # 优化后：按72DPI换算，600px≈212mm，刚好匹配A4纸宽度（210mm）
IMAGE_QUALITY = 100  # 保持不变（确保图像清晰度）
# WHITE_DOC_LIST = ['05-03-18-political-release']
# ratio = "text_tokens" / (1024/16)*(1024/16)/16
LIMIT_NUM = 1                     # 限制处理pdf的数量，方便小规模验证

def set_env():
    """解决root用户XDG_RUNTIME_DIR未设置的警告，避免潜在环境错误"""
    if os.geteuid() == 0:  # 检测是否为root用户
        os.environ.setdefault("XDG_RUNTIME_DIR", "/tmp/runtime-root")
        os.makedirs(os.environ["XDG_RUNTIME_DIR"], exist_ok=True)
        os.chmod(os.environ["XDG_RUNTIME_DIR"], 0o700)  # 设置权限
        print(f"⚠️  已为root用户设置XDG_RUNTIME_DIR：{os.environ['XDG_RUNTIME_DIR']}")

    # 创建数据目录（如果不存在）
    os.makedirs(DATASET_PDF_PATH, exist_ok=True)
    os.makedirs(OUTPUT_PDF, exist_ok=True)
    os.makedirs(RESIZED_IMGAE_PATH, exist_ok=True)
    os.makedirs(MARKDOWN_OUTPUT, exist_ok=True)
    print(f"✅ 初始化数据集pdf路径：{DATASET_PDF_PATH}")
    print(f"✅ 初始化图像输出路径：{OUTPUT_PDF}")
    print(f"✅ 初始化resize图像路径：{RESIZED_IMGAE_PATH}")
    print(f"✅ 初始化图像输出目录：{MARKDOWN_OUTPUT}")

In [57]:
# 处理pdf目录下的所有pdf文件
def process_all_pdf_to_image(pdf_dir):
    if not os.path.exists(pdf_dir) or not os.path.isdir(pdf_dir):
        print(f"错误：目录不存在或不是文件夹 → {pdf_dir}")
    else:
        # 遍历目录，筛选PDF文件
        processed_pdf_file_num = 0
        pdf_files_info = []  # 存储 (无扩展名文件名, 完整路径) 列表
        for file_name in os.listdir(pdf_dir):
            if processed_pdf_file_num >= LIMIT_NUM:  # 限制处理pdf的数量
                break
            if not file_name.lower().endswith(".pdf"):  # 跳过非pdf文件名的文件
                continue
            # 拼接文件完整路径
            file_path = os.path.join(pdf_dir, file_name)
            # 筛选：仅保留文件（排除子目录）+ 后缀为 .pdf（忽略大小写）
            if os.path.isfile(file_path) and file_name.lower().endswith(".pdf"):
                # 提取无扩展名文件名（核心：分割文件名和扩展名）
                file_name_without_ext = os.path.splitext(file_name)[0]
                pdf_files_info.append((file_name_without_ext, file_path))
                # 在resized_image/目录下创建同名子目录，用于存放同名pdf的拆分后的页面图像
                resized_image_subdir_path = os.path.join(RESIZED_IMGAE_PATH, file_name_without_ext)
                os.makedirs(resized_image_subdir_path, exist_ok=True)
            process_pdf_ocr(file_path, resized_image_subdir_path)
            processed_pdf_file_num = processed_pdf_file_num + 1

        if not pdf_files_info:
            print(f"提示：目录 {pdf_dir} 下未找到PDF文件")
        else:
            print(f"成功找到 {len(pdf_files_info)} 个PDF文件：")
            print("-" * 50)
            for name, path in pdf_files_info:
                print(f"文件名：{name}")
                print(f"完整路径：{path}")
                print("-" * 30)

# 对于每一个pdf文件进行拆分页面，并resize+padding到目标分辨率
def process_pdf_ocr(pdf_file, output_path):
    # 1. 确保输出目录存在（不存在则创建，避免保存报错）
    os.makedirs(output_path, exist_ok=True)

    # 2. PDF转图像
    pdf_images = pdf_to_images(pdf_file)
    original_sizes = [img.size for img in pdf_images]  # 保存原尺寸（width, height）

    # 3. 逐页resize
    for page_idx, (img, original_size) in enumerate(zip(pdf_images, original_sizes)):
        print(f"\n处理第{page_idx+1}/{len(pdf_images)}页...")

        # 3.1 计算Resize+Padding参数
        resize_size, padding, actual_tokens = calculate_resize_padding_for_target_resolution(
            original_size=original_size,
            patch_size=PATCH_SIZE,
            resolution=TARGET_RESIZE_IMAGE_RESOLUTION,
        )

        # 3.2 Resize图像（保持宽高比，LANCZOS插值保证清晰度)
        resized_img = img.resize(resize_size, Image.LANCZOS)

        # 3.3 Padding至1024×1024（白色背景，图像居中）
        target_img = Image.new("RGB", (1024, 1024), color="white")  # 创建目标尺寸画布
        pad_left, pad_top, _, _ = padding
        target_img.paste(resized_img, (pad_left, pad_top))  # 粘贴图像到居中位置
        print(f"原尺寸：{original_size} → Resize尺寸：{resize_size} → Padding：{padding} → 最终尺寸：{target_img.size} → 实际Tokens：{actual_tokens}")

        # 3.4 拼接目标文件路径，检查是否已存在（核心：不覆盖，只追加）
        img_filename = f"page_{page_idx+1}.jpg"
        path = os.path.join(output_path, img_filename)
        if os.path.exists(path):
            # 已存在同名文件，跳过保存
            print(f"✅ 已存在文件：{img_filename}，跳过保存（不覆盖）")
            continue

        # 3.5 不存在则保存（修正原代码bug：保存padding后的target_img，而非resized_img）
        target_img.save(path, quality=100)
        print(f"📥 新增保存文件：{path}，尺寸：{target_img.size}")

# 行业标准： 300dpi 高分辨率渲染 → 再 resize/padding 到目标尺寸
def pdf_to_images(pdf_path, dpi=300):
    """将PDF每页转为PIL图像（dpi控制原始分辨率）"""
    try:
        images = convert_from_path(
            pdf_path,
            dpi=dpi,
            fmt="jpg",
            thread_count=16,
            use_cropbox=True,
            strict=False
        )
        print(f"成功读取PDF：{pdf_path}，共{len(images)}页")
        return images
    except Exception as e:
        print(f"PDF转图像失败：{e}")
        raise

# 固定目标1024×1024，保持长宽比
def calculate_resize_padding_for_target_resolution(original_size, patch_size=16, resolution=1024):
    """
    保持长宽比不变，计算图像Resize+Padding的参数，最终分辨率固定为1024×1024
    Args:
        original_size: (width, height) 原图像尺寸（PIL.Image.size返回格式）
        patch_size: 模型Patch Size（DeepSeek-OCR默认16，无需修改）
    Returns:
        resize_size: (new_width, new_height) Resize后的尺寸（保持长宽比，Patch Size整数倍）
        padding: (pad_left, pad_top, pad_right, pad_bottom) 上下左右补白量（最终凑1024×1024）
        actual_tokens: 实际Vision Tokens长度（(1024/patch_size) × (1024/patch_size) = 4096）
    """
    target_resolution = (resolution, resolution)  # 固定目标分辨率：1024×1024
    original_w, original_h = original_size
    aspect_ratio = original_w / original_h  # 原始宽高比（保持不变）

    # 第一步：计算Resize尺寸（保持长宽比，不超过1024×1024，且为Patch Size整数倍）
    scale = min(
        target_resolution[0] / original_w,
        target_resolution[1] / original_h
    )
    resize_w = int(original_w * scale)
    resize_h = int(original_h * scale)
    # 调整为Patch Size的整数倍（确保模型编码时Tokens无残留）
    resize_w = (resize_w // patch_size) * patch_size
    resize_h = (resize_h // patch_size) * patch_size
    # 边界修正：避免极端小图像导致尺寸为0
    resize_w = max(resize_w, patch_size)
    resize_h = max(resize_h, patch_size)

    # 第二步：计算Padding参数（上下左右补白，图像居中，最终凑1024×1024）
    pad_total_w = target_resolution[0] - resize_w  # 横向总补白量
    pad_total_h = target_resolution[1] - resize_h  # 纵向总补白量
    # 对称补白（图像居中显示）
    pad_left = pad_total_w // 2
    pad_right = pad_total_w - pad_left
    pad_top = pad_total_h // 2
    pad_bottom = pad_total_h - pad_top

    # 第三步：计算固定Vision Tokens数量（1024/Patch Size 为固定值）
    patch_h = target_resolution[1] // patch_size
    patch_w = target_resolution[0] // patch_size
    actual_tokens = patch_h * patch_w

    return (resize_w, resize_h), (pad_left, pad_top, pad_right, pad_bottom), actual_tokens

In [58]:
"""
    目录结构：
        resized_image/a/page_1.jpg
        resized_image/a/page_2.jpg
        resized_image/b/page_1.jpg
        resized_image/b/page_2.jpg
        resized_image/b/page_3.jpg
        resized_image/c/page_1.jpg
        ...
"""
def convert_all_image_to_markdown(all_pdf_images_dir, output_dir):
    all_pdf_images_dir = Path(all_pdf_images_dir)
    output_dir = Path(output_dir)

    # 1. 遍历主输入目录下的所有子目录，即pdf同名目录(a、b、c...)
    images_input_subdirs = [subdir for subdir in all_pdf_images_dir.iterdir() if subdir.is_dir()]
    if not images_input_subdirs:
        print(f"⚠️  警告：主输入目录 {all_pdf_images_dir.name} 下无任何子目录")
        return

    # 2. 逐个处理子目录
    for idx, images_input_subdir in enumerate(images_input_subdirs, 1):
        if idx > LIMIT_NUM:  # 限制处理pdf image的数量
            break
        subdir_name = images_input_subdir.name  # 子目录名称（如"subdir_1"）
        print(f"\n📌 处理第{idx}/{len(images_input_subdirs)}个输入子目录：{subdir_name}")
        # if subdir_name not in WHITE_DOC_LIST:
        #     print(f"{subdir_name} not in white_doc_list")
        #     continue

        # 3. 创建对应的输出子目录（markdown_output_subdirs）
        # 输出子目录路径：output_dir / 输入子目录名（与输入子目录同名，避免冲突）
        markdown_output_subdir = output_dir / subdir_name
        markdown_output_subdir.mkdir(exist_ok=True)  # 不存在则创建

        # 4. 遍历当前输入子目录下的所有jpg文件（支持大小写）
        jpg_files = list(images_input_subdir.glob(f"*.jpg")) # 仅匹配jpg文件
        jpg_files = [f for f in jpg_files if f.is_file()]    # 二次筛选文件（避免目录）
        page_num = len(jpg_files)

        if not jpg_files:
            print(f"  ⚠️  警告：该子目录下无任何jpg文件，跳过函数调用")
            print("-" * 30)
            continue

        # 5. 调用转换函数，转为markdown(mmd文件)
        try:
            convert_images_to_markdown(images_input_subdir, markdown_output_subdir, page_num)
        except Exception as e:
            print(f"  ❌ 函数调用失败：{str(e)}")
        print("-" * 30)

    print("\n🎉 所有子目录处理完毕！")
    print(f"📊 统计：共处理 {len(images_input_subdirs)} 个输入子目录")
    print(f"📁 输出文件存放位置：{output_dir.resolve()}")

def convert_images_to_markdown(images_input_path, output_dir, page_num):
    for idx in range(page_num):
        img_path = os.path.join(images_input_path, "page_" + str(idx+1) + ".jpg")
        output_file_path = os.path.join(output_dir,"page_"+str(idx+1))
        convert_img_to_markdown(img_path, output_file_path)

def convert_img_to_markdown(image_file, output_dir):
    output_dir = Path(output_dir)
    if os.path.exists(output_dir):
        # 已存在同名目录，跳过保存
        print(f"✅ 已存在目录：{output_dir.name}，跳过转换markdown（不覆盖）")
        return
    os.makedirs(output_dir, exist_ok=True)
    prompt = "<image>\n<|grounding|>Convert the document to markdown. extract all details. "
    # 调用模型infer生成Markdown（复用原逻辑）
    model.infer(
        tokenizer=tokenizer,
        prompt=prompt,
        image_file=image_file,
        output_path=output_dir,
        base_size=BASE_SIZE,
        image_size=IMAGE_SIZE,
        crop_mode=CROP_MODE,            # True
        save_results=SAVE_OCR_RESULTS,  # True
        test_compress=TEST_COMPRESS,    # True
    )

    # 释放未使用的GPU显存
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    print("✅ markdown生成完成！")

In [59]:
def convert_all_markdown_to_pdf(all_markdown_dir):
    all_markdown_dir = Path(all_markdown_dir)
    # 1. 遍历all_markdown_dir目录下的所有子目录，即pdf同名目录(a、b、c...)
    markdown_input_subdirs = [subdir for subdir in all_markdown_dir.iterdir() if subdir.is_dir()]
    if not markdown_input_subdirs:
        print(f"⚠️  警告：主输入目录 {all_markdown_dir.name} 下无任何子目录")
        return

    # 2. 逐个处理子目录
    for idx, markdown_input_subdir in enumerate(markdown_input_subdirs, 1):
        if idx > LIMIT_NUM:  # 限制处理pdf image的数量
            break
        subdir_name = markdown_input_subdir.name  # 子目录名称（如"subdir_1"）
        print(f"\n📌 处理第{idx}/{len(markdown_input_subdirs)}个输入子目录：{subdir_name}")
        # if subdir_name not in WHITE_DOC_LIST:
        #     print(f"{subdir_name} not in white_doc_list")
        #     continue

        # 3. 遍历当前输入子目录下的所有子目录(a/page_1/...)
        subdir_pages = [
            subdir for subdir in markdown_input_subdir.iterdir()
            if subdir.is_dir() and subdir.name.startswith("page_")
        ]
        subdir_pages.sort(key=lambda x: int(x.name.split("_")[1]))  # 按页面编号升序排序（关键）
        page_num = len(subdir_pages)
        if not subdir_pages:
            print(f"  ⚠️  警告：该子目录下无任何page_*子目录，跳过函数调用")
            print("-" * 30)
            continue

        # 4. 调用mmd2pdf转换函数，将markdown文件转为pdf
        try:
            # Markdown子目录路径（用于找result.mmd）
            markdown_subdir_path = str(markdown_input_subdir)
            # 最终PDF输出路径（保存到OUTPUT_PDF，带.pdf后缀）
            final_pdf_path = os.path.join(OUTPUT_PDF, f"{subdir_name}.pdf")
            # 传递两个路径给mmd2pdf
            mmd2pdf(markdown_subdir_path, final_pdf_path, page_num, subdir_name)
            print(f"  ✅ 函数调用成功：mmd2pdf({final_pdf_path}, {page_num})")
        except Exception as e:
            print(f"  ❌ 函数调用失败：{str(e)}")
        print("-" * 30)

def mmd2pdf(markdown_subdir_path, final_pdf_path, page_num, pdf_name):
    # 检查最终PDF是否已存在
    if os.path.exists(final_pdf_path):
        print(f"✅ 已存在PDF：{os.path.basename(final_pdf_path)}，跳过转换（不覆盖）")
        return

    image_paths = []
    for page_idx in range(1, page_num+1):
        # 正确的page_dir路径：markdown_subdir_path/page_xxx
        page_dir = os.path.join(markdown_subdir_path, f"page_{page_idx}")
        print(f"\n--- 处理第{page_idx}/{page_num}页（{page_dir}）---")
        img_path = mmd_to_image(str(page_dir), page_idx, pdf_name)
        image_paths.append(img_path)

    # 合并PDF到最终输出路径（带.pdf后缀）
    print("\n" + "="*50)
    images_to_pdf(image_paths, final_pdf_path)

    # 输出总结（修正显示路径）
    print("\n" + "="*50)
    print("📊 流程总结：")
    print(f" - 总页面数{page_num}")
    print(f" - 成功转换：{len([x for x in image_paths if x])}页")
    print(f" - 图像目录：{os.path.abspath(os.path.join(OUTPUT_PDF, pdf_name))}")
    print(f" - 最终PDF：{os.path.abspath(final_pdf_path)}")
    print("="*50)

# todo 检查是否正确
def mmd_to_image(page_dir, page_num, pdf_name):
    mmd_path = os.path.join(page_dir, "result.mmd")
    images_dir = os.path.join(page_dir, "images")

    if not os.path.exists(mmd_path):
        print(f"❌ 第{page_num}页：未找到result.mmd，跳过")
        return None
    if not os.path.exists(images_dir):
        print(f"❌ 第{page_num}页：未找到images文件夹，跳过")
        return None

    # 1. 读取并修复路径（添加file://）
    with open(mmd_path, "r", encoding="utf-8") as f:
        mmd_content = f.read()
    fixed_mmd = fix_and_check_image_paths(mmd_content, images_dir)

    # 2. 转换为HTML
    html_content = markdown2.markdown(
        fixed_mmd,
        extras=["fenced-code-blocks", "tables", "footnotes"]
    )

    # 3. 构建HTML（简化，避免解析问题）
    full_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            /* 核心：基础排版配置（解决文本极小问题） */
            html, body {{
                margin: 0 !important;
                padding: 15px !important;  /* 给内容留少量内边距，避免贴边 */
                width: 100% !important;
                max-width: 100% !important;
                box-sizing: border-box;
                background-color: white;
                font-family: Arial, "SimHei", "Microsoft YaHei", sans-serif;  /* 支持中文显示 */
                font-size: 16px !important;  /* 默认字体大小16px（清晰可见） */
                line-height: 1.6 !important;  /* 行高1.6，文本不拥挤 */
                color: #333 !important;  /* 文本颜色加深，提高可读性 */
            }}
            /* 图像优化：确保图像不小于页面宽度，且不模糊 */
            img {{
                max-width: 100% !important;
                height: auto !important;
                display: block !important;
                margin: 10px auto !important;  /* 图像上下留空，居中显示 */
                min-width: 80% !important;  /* 避免图像过小 */
                border: none !important;
            }}
            /* 段落优化：上下留间距，避免拥挤 */
            p {{
                margin: 10px 0 !important;
                text-align: justify !important;  /* 文本两端对齐，布局更整齐 */
            }}
            /* 标题优化：区分层级，大小适中 */
            h1 {{ font-size: 24px !important; margin: 20px 0 !important; }}
            h2 {{ font-size: 22px !important; margin: 18px 0 !important; }}
            h3 {{ font-size: 20px !important; margin: 16px 0 !important; }}
            /* 表格优化：边框清晰，自适应宽度 */
            table {{
                width: 100% !important;
                border-collapse: collapse !important;
                margin: 15px 0 !important;
            }}
            td, th {{
                border: 1px solid #666 !important;
                padding: 8px 12px !important;
                text-align: center !important;
                word-break: break-all !important;  /* 表格文本换行，避免溢出 */
            }}
            /* 代码块优化：避免溢出，背景区分 */
            pre {{
                background-color: #f5f5f5 !important;
                padding: 10px !important;
                border-radius: 4px !important;
                white-space: pre-wrap !important;
                word-break: break-all !important;
                font-size: 14px !important;
            }}
            /* 列表优化：缩进合理 */
            ul, ol {{
                margin: 10px 0 !important;
                padding-left: 30px !important;
            }}
        </style>
    </head>
    <body>
        <div class="content-container">{html_content}</div>
    </body>
    </html>
    """

    # 4. 保存临时HTML
    temp_html_path = os.path.join(page_dir, f"temp_page_{page_num}.html")
    with open(temp_html_path, "w", encoding="utf-8") as f:
        f.write(full_html)

    # 5. 关键参数（添加--enable-local-file-access，Ubuntu 0.12.6支持）
    # 创建PDF专属图像目录
    pdf_img_dir = os.path.join(OUTPUT_PDF, pdf_name)
    os.makedirs(pdf_img_dir, exist_ok=True)
    # 图像保存到专属目录
    img_output_path = os.path.join(pdf_img_dir, f"page_{page_num}.png")

    # 配置
    imgkit_options = {
        "format": "png",
        "width": IMAGE_WIDTH,
        "quality": IMAGE_QUALITY,
        "encoding": "UTF-8",
        "quiet": "",  # 隐藏冗余输出
        "enable-local-file-access": "",  # 允许本地文件访问（解决协议错误的核心！）
    }
    config = imgkit.config(wkhtmltoimage=WKHTMLTOIMAGE_PATH)

    # 执行转换（捕获详细错误）
    try:
        # 直接调用subprocess执行命令（绕过imgkit可能的参数传递问题）
        cmd = [
            WKHTMLTOIMAGE_PATH,
            "--format", "png",
            "--width", str(IMAGE_WIDTH),
            "--height", "900",  # 新增：设置高度（A4纸高度297mm≈842px，留冗余）
            "--quality", str(IMAGE_QUALITY),
            "--encoding", "UTF-8",
            "--quiet",
            "--enable-local-file-access",
            temp_html_path,
            img_output_path
        ]

        print(WKHTMLTOIMAGE_PATH)
        print(temp_html_path)
        print(img_output_path)
        # 执行命令并捕获输出
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            env=os.environ  # 传递修复后的环境变量
        )

        # 验证转换结果
        if os.path.exists(img_output_path) and os.path.getsize(img_output_path) > 2048:
            print(f"✅ 第{page_num}页：转换成功 → {os.path.basename(img_output_path)}")
            return img_output_path
        else:
            print(f"❌ 第{page_num}页：转换失败")
            print(f"  命令输出：{result.stdout}")
            print(f"  错误信息：{result.stderr}")
            return None
    except Exception as e:
        print(f"❌ 第{page_num}页：报错 → {str(e)}")
        return None
    finally:
        # 强制删除临时文件
        if os.path.exists(temp_html_path):
            os.remove(temp_html_path)

def images_to_pdf(image_paths, pdf_output):
    valid_images = [
        img for img in image_paths
        if img and os.path.exists(img) and os.path.getsize(img) > 2048
    ]

    if not valid_images:
        print("\n❌ 无有效图像可合并，PDF生成失败")
        return

    valid_images.sort(key=lambda x: int(os.path.basename(x).split("_")[1].split(".")[0]))
    print(f"\n📊 共{len(valid_images)}页有效图像，合并PDF中...")

    pdf = FPDF(unit="mm", format="A4")
    pdf.set_auto_page_break(auto=True, margin=10)

    for img_path in valid_images:
        try:
            with Image.open(img_path) as img:
                img_w_px, img_h_px = img.size

             # 保持72DPI换算（与wkhtmltoimage旧版本默认值一致）
            DPI = 72  
            img_w_mm = (img_w_px / DPI) * 25.4
            img_h_mm = (img_h_px / DPI) * 25.4

            # 优化：减小边距（从20mm改为10mm），扩大有效显示区域
            max_w = 210 - 10  # A4宽度210mm，预留10mm边距（左右各5mm）
            max_h = 297 - 10  # A4高度297mm，预留10mm边距（上下各5mm）
            scale = min(max_w / img_w_mm, max_h / img_h_mm, 1.0)  # 最大缩放比例不超过1.0（避免拉伸模糊）

            pdf.add_page()
            pdf.image(
                img_path,
                x=(210 - img_w_mm * scale) / 2,  # 水平居中
                y=5,  # 顶部边距5mm
                w=img_w_mm * scale,
                h=img_h_mm * scale
            )
        except Exception as e:
            print(f"⚠️  跳过异常图像：{os.path.basename(img_path)} → {str(e)}")

    # os.makedirs(os.path.join(OUTPUT_PDF, ), exist_ok=True)
    pdf.output(pdf_output)
    print(f"\n🎉 PDF合并完成！路径：{os.path.abspath(pdf_output)}")

def fix_and_check_image_paths(mmd_content, images_dir):
    """
    关键修复：Ubuntu下必须显式指定file://协议，格式为 file:///绝对路径（三个斜杠）
    例：/home/root/image_output/page_1/images/0.jpg → file:///home/root/image_output/page_1/images/0.jpg
    """
    image_pattern = r"!\[(.*?)\]\((images/[^)]+)\)"
    missing_images = []

    def replace_and_check(match):
        desc = match.group(1)
        rel_path = match.group(2)
        img_filename = os.path.basename(rel_path)
        abs_path = os.path.abspath(os.path.join(images_dir, img_filename))

        # 验证图片存在性
        if not os.path.exists(abs_path) or os.path.getsize(abs_path) < 1024:
            missing_images.append(abs_path)
            return f"![{desc}（图片缺失）]({abs_path})"

        # 显式添加file://协议（Ubuntu 0.12.6必需）
        file_url = f"file://{abs_path}"  # 注意：Ubuntu下是file://+绝对路径（自动转为三个斜杠）
        return f"![{desc}]({file_url})"

    fixed_mmd = re.sub(image_pattern, replace_and_check, mmd_content)
    if missing_images:
        print(f"⚠️  缺失图片：{missing_images}")
    return fixed_mmd

In [60]:
def calculate_pdf_tokens(pdf_markdown_dir, tokenizer, resolution=TARGET_RESIZE_IMAGE_RESOLUTION):
    """
    统计单个PDF的文本tokens和vision tokens
    Args:
        pdf_markdown_dir: 单个PDF对应的Markdown输出子目录（如 markdown_output/xxx）
        tokenizer: 模型Tokenizer（复用全局已加载的tokenizer）
        resolution: 目标分辨率（默认1024）
    Returns:
        dict: 统计结果（PDF名称、页面数、文本tokens、vision tokens、比值）
    """
    pdf_name = pdf_markdown_dir.name
    # 1. 统计页面数（page_*子目录数量）
    page_subdirs = [subdir for subdir in pdf_markdown_dir.iterdir() if subdir.is_dir() and subdir.name.startswith("page_")]
    page_num = len(page_subdirs)
    if page_num == 0:
        print(f"⚠️  {pdf_name} 无有效页面子目录，跳过统计")
        return None

    # 2. 计算vision tokens（按用户给定公式：(resolution/16)*(resolution/16)/16 * page_num）
    patch_side = resolution // PATCH_SIZE  # 1024/16=64
    vision_tokens_per_page = (patch_side * patch_side) // 16  # (64*64)/16=256
    total_vision_tokens = vision_tokens_per_page * page_num

    # 3. 计算文本tokens（读取所有page的result.mmd，合并后编码）
    total_text = ""
    for page_subdir in page_subdirs:
        mmd_file = page_subdir / "result.mmd"
        if not mmd_file.exists():
            print(f"⚠️  {pdf_name} - {page_subdir.name} 缺失result.mmd，跳过该页面文本")
            continue
        # 读取Markdown文本（忽略编码错误）
        try:
            with open(mmd_file, "r", encoding="utf-8", errors="ignore") as f:
                page_text = f.read().strip()
            total_text += page_text + "\n"  # 页面间换行分隔
        except Exception as e:
            print(f"⚠️  读取 {mmd_file} 失败：{str(e)}，跳过该页面文本")

    # 编码文本得到tokens数量（不添加special tokens）
    if total_text.strip():
        text_tokens = len(tokenizer.encode(total_text, add_special_tokens=False))
    else:
        text_tokens = 0
        print(f"⚠️  {pdf_name} 无有效文本内容，文本tokens为0")

    # 4. 计算文本tokens/vision tokens比值
    token_ratio = text_tokens / total_vision_tokens if total_vision_tokens != 0 else 0.0

    return {
        "pdf_name": pdf_name,
        "page_num": page_num,
        "text_tokens": text_tokens,
        "vision_tokens": total_vision_tokens,
        "text_vision_ratio": round(token_ratio, 4)
    }

def stat_all_pdf_tokens(markdown_root_dir, output_stat_path="pdf_tokens_stat.csv"):
    """
    统计所有PDF的tokens信息（入口函数）
    Args:
        markdown_root_dir: Markdown输出根目录（MARKDOWN_OUTPUT）
        output_stat_path: 统计结果保存路径（CSV文件）
    """
    markdown_root_dir = Path(markdown_root_dir)
    if not markdown_root_dir.exists():
        print(f"❌ Markdown根目录不存在：{markdown_root_dir}")
        return

    # 1. 遍历所有PDF对应的Markdown子目录
    pdf_markdown_dirs = [subdir for subdir in markdown_root_dir.iterdir() if subdir.is_dir()]
    if not pdf_markdown_dirs:
        print(f"⚠️  Markdown根目录下无有效PDF子目录：{markdown_root_dir}")
        return

    # 2. 逐个统计每个PDF的tokens
    all_stat_results = []
    print("\n" + "="*80)
    print("📊 开始统计所有PDF的Tokens信息")
    print("="*80)
    for idx, pdf_dir in enumerate(pdf_markdown_dirs, 1):
        if idx > LIMIT_NUM:  # 遵循原代码的数量限制
            break
        print(f"\n[{idx}/{len(pdf_markdown_dirs)}] 统计PDF：{pdf_dir.name}")
        stat_result = calculate_pdf_tokens(pdf_dir, tokenizer)
        if stat_result:
            all_stat_results.append(stat_result)
            # 打印单PDF统计结果
            print(f"  - 页面数：{stat_result['page_num']}")
            print(f"  - 文本Tokens：{stat_result['text_tokens']}")
            print(f"  - Vision Tokens：{stat_result['vision_tokens']}")
            print(f"  - 文本Tokens/Vision Tokens：{stat_result['text_vision_ratio']}")

    # 3. 保存统计结果到CSV文件
    if all_stat_results:
        # 确保输出目录存在
        output_path = Path(output_stat_path)
        output_path.parent.mkdir(exist_ok=True)
        # 写入CSV
        with open(output_path, "w", encoding="utf-8", newline="") as f:
            fieldnames = ["pdf_name", "page_num", "text_tokens", "vision_tokens", "text_vision_ratio"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(all_stat_results)
        print(f"\n" + "="*80)
        print(f"✅ 统计结果已保存到：{output_path.resolve()}")

        # 打印汇总信息
        total_pdf = len(all_stat_results)
        avg_text_tokens = sum([r["text_tokens"] for r in all_stat_results]) / total_pdf
        avg_vision_tokens = sum([r["vision_tokens"] for r in all_stat_results]) / total_pdf
        avg_ratio = sum([r["text_vision_ratio"] for r in all_stat_results]) / total_pdf
        print(f"\n📈 汇总统计（共{total_pdf}个PDF）：")
        print(f"  - 平均页面数：{round(sum([r['page_num'] for r in all_stat_results])/total_pdf, 1)}")
        print(f"  - 平均文本Tokens：{round(avg_text_tokens, 0)}")
        print(f"  - 平均Vision Tokens：{round(avg_vision_tokens, 0)}")
        print(f"  - 平均文本/Vision比值：{round(avg_ratio, 4)}")
    else:
        print(f"\n❌ 无有效统计结果")
    print("="*80)

In [61]:
"""
    process_all_pdf: PDF -> images -> resize+padding(images) -> images(1024*1024)
    convert_all_pdf: images(1024*1024) -- DPOCR --> MarkDown -> PDF
"""
if __name__ == "__main__":
    # 初始化环境
    set_env()

    # 执行主流程
    process_all_pdf_to_image(DATASET_PDF_PATH)  # 把MMLongBench-Doc的所有pdf文档逐页resize到resized_image/目录的同名子目录下，以1024*1024分辨率图像存放
    convert_all_image_to_markdown(RESIZED_IMGAE_PATH, MARKDOWN_OUTPUT)  # 把temp/目录下所有子目录的图像，通过DeepSeek-ocr转为markdown文件
    convert_all_markdown_to_pdf(MARKDOWN_OUTPUT) # 把所有提取后的markdown添加html布局，合并转为pdf

    # 新增：统计每个PDF的文本tokens和vision tokens
    stat_all_pdf_tokens(MARKDOWN_OUTPUT, output_stat_path="pdf_tokens_stat.csv")

⚠️  已为root用户设置XDG_RUNTIME_DIR：/tmp/runtime-root
✅ 初始化数据集pdf路径：data/MMLongBench-Doc/data/documents/
✅ 初始化图像输出路径：pdf_output/
✅ 初始化resize图像路径：resized_image/
✅ 初始化图像输出目录：markdown_output/

📌 处理第1/1个输入子目录：PWC_opportunity_of_lifetime

--- 处理第1/18页（markdown_output/PWC_opportunity_of_lifetime/page_1）---
wkhtmltoimage
markdown_output/PWC_opportunity_of_lifetime/page_1/temp_page_1.html
pdf_output/PWC_opportunity_of_lifetime/page_1.png
✅ 第1页：转换成功 → page_1.png

--- 处理第2/18页（markdown_output/PWC_opportunity_of_lifetime/page_2）---
wkhtmltoimage
markdown_output/PWC_opportunity_of_lifetime/page_2/temp_page_2.html
pdf_output/PWC_opportunity_of_lifetime/page_2.png
✅ 第2页：转换成功 → page_2.png

--- 处理第3/18页（markdown_output/PWC_opportunity_of_lifetime/page_3）---
wkhtmltoimage
markdown_output/PWC_opportunity_of_lifetime/page_3/temp_page_3.html
pdf_output/PWC_opportunity_of_lifetime/page_3.png
✅ 第3页：转换成功 → page_3.png

--- 处理第4/18页（markdown_output/PWC_opportunity_of_lifetime/page_4）---
wkhtmltoimage
markdown_output